In [1]:
# Bewerten von Filmkritiken (1 Positiv, 0 Negativ)

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

# Warum auch immer, bekomme ich einen fehler wenn ich das hier nicht drinne habe. 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
train_data, validation_data, test_data = tfds.load(name='imdb_reviews', split=('train[:60%]', 'train[:60%]', 'test'), as_supervised=True)

In [3]:
train_examples_batch , train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [5]:
# Das Modell muss drei wichtige Teile beachten
# Wie kann man den Text darstellen?
# Wie viele Ebenen?
# Wie viele versteckte pro Layer?

# Hier sätze mit Ergebnis 0 oder 1
# Sätze darstellen durch Embeddings

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer) # Das reicht, um das Embedding zu verwenden.
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Verschiedene Loss-functions möglich, z.B. MSE aber binary_crossentropy ist besser da der "Abstand" zwischen Wahrscheinlichkeitsverteilungen gemessen wird
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [8]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=20, validation_data=validation_data.batch(512), verbose=1)

results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

Epoch 1/20
30/30 [==============================] - 2s 60ms/step - loss: 0.8465 - accuracy: 0.5269 - val_loss: 0.7129 - val_accuracy: 0.5877
Epoch 2/20
30/30 [==============================] - 2s 52ms/step - loss: 0.6508 - accuracy: 0.6299 - val_loss: 0.6019 - val_accuracy: 0.6632
Epoch 3/20
30/30 [==============================] - 2s 52ms/step - loss: 0.5779 - accuracy: 0.6807 - val_loss: 0.5498 - val_accuracy: 0.6931
Epoch 4/20
30/30 [==============================] - 2s 52ms/step - loss: 0.5276 - accuracy: 0.7209 - val_loss: 0.4992 - val_accuracy: 0.7373
Epoch 5/20
30/30 [==============================] - 2s 52ms/step - loss: 0.4791 - accuracy: 0.7590 - val_loss: 0.4535 - val_accuracy: 0.7679
Epoch 6/20
30/30 [==============================] - 2s 52ms/step - loss: 0.4361 - accuracy: 0.7863 - val_loss: 0.4136 - val_accuracy: 0.8161
Epoch 7/20
30/30 [==============================] - 2s 52ms/step - loss: 0.3967 - accuracy: 0.8153 - val_loss: 0.3755 - val_accuracy: 0.8261
Epoch 8/20
30